In [4]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model
import keras
import cv2

In [5]:
# Каталог с данными для обучения
train_dir = 'train2'
# Каталог с данными для проверки
val_dir = 'val2'
# Каталог с данными для тестирования
test_dir = 'test2'
# Размеры изображения
img_width, img_height = 224, 224
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 1400
# Количество изображений для проверки
nb_validation_samples = 300
# Количество изображений для тестирования
nb_test_samples = 300

In [6]:
model = keras.applications.mobilenet.MobileNet(input_shape=input_shape, alpha=1.0, depth_multiplier=1, dropout=1e-3, include_top=True, weights=None, input_tensor=None, pooling=None, classes=2)

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [9]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    classes=['0','1'],
    batch_size=batch_size,
    class_mode='categorical')

Found 2789 images belonging to 2 classes.


In [10]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    classes=['0','1'],
    batch_size=batch_size,
    class_mode='categorical')

Found 598 images belonging to 2 classes.


In [11]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    classes=['0','1'],
    batch_size=batch_size,
    class_mode='categorical')

Found 597 images belonging to 2 classes.


In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save("model_v2.h5")

Epoch 1/10
87/87 [==============================] - 59s 673ms/step - loss: 0.4947 - acc: 0.7795 - val_loss: 0.2919 - val_acc: 0.9097
Epoch 2/10
87/87 [==============================] - 45s 516ms/step - loss: 0.1832 - acc: 0.9274 - val_loss: 0.2661 - val_acc: 0.9167
Epoch 3/10
87/87 [==============================] - 46s 525ms/step - loss: 0.1207 - acc: 0.9590 - val_loss: 0.4312 - val_acc: 0.8237
Epoch 4/10
87/87 [==============================] - 44s 501ms/step - loss: 0.0918 - acc: 0.9677 - val_loss: 0.1283 - val_acc: 0.9583
Epoch 5/10
87/87 [==============================] - 43s 497ms/step - loss: 0.0603 - acc: 0.9763 - val_loss: 0.4214 - val_acc: 0.8741
Epoch 6/10
87/87 [==============================] - 44s 500ms/step - loss: 0.0519 - acc: 0.9813 - val_loss: 0.1918 - val_acc: 0.9271
Epoch 7/10
87/87 [==============================] - 43s 497ms/step - loss: 0.0766 - acc: 0.9720 - val_loss: 0.1200 - val_acc: 0.9676
Epoch 8/10
87/87 [==============================] - 42s 488ms/step - 

In [13]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


In [14]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 98.26%


In [3]:
model = load_model('model_v1.h5')

In [29]:
img = cv2.imread('test2/1/image.jpg')
#cv2.imshow("img", img)
#cv2.waitKey(0) & 0xFF
resized_img = cv2.resize(img, (224, 224))
print(resized_img.shape)
img = resized_img.reshape(1,224,224,3)
img = img.astype('float32')
img /= 255
preds = model.predict(img)
labels = [0, 1]
ind = preds[0].argmax()
print(preds)
import numpy as np

if round(preds[0][ind]*100,2)>75:
    print("Ответ : ",labels[ind], ", точность :", round(preds[0][ind]*100,2),"%" )
else:
    print("объект не распознан! Преблизительно - ", labels[ind], "точность :", round(preds[0][ind]*100,2),"%" ) 

(224, 224, 3)
[[0.09174645 0.90825355]]
Ответ :  1 , точность : 90.83 %


In [1]:
def recognition(img):
    resized_img = cv2.resize(img, (224, 224))
    print(resized_img.shape)
    img = resized_img.reshape(1,224,224,3)
    img = img.astype('float32')
    img /= 255
    preds = model.predict(img)
    labels = [0,1]
    ind = preds[0].argmax()
    print("Ответ : ",labels[ind], ", точность :", round(preds[0][ind]*100,2),"%" )
    acc = round(preds[0][ind]*100,2)
    if acc > 75:
        res = ""+str(labels[ind])+", "+str(acc) +"%"
    else:
        res= "0, 0%"
    return res

In [ ]:
import cv2 
import os 

cam = cv2.VideoCapture(0) 
cam.set(3, 640) # set video width 
cam.set(4, 480) # set video height 

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 

# For each person, enter one numeric face id 
face_id = input('\n enter user id end press ==> ') 

print("\n [INFO] Initializing face capture. Look the camera and wait ...") 
# Initialize individual sampling face count 
count = 0 

while(True): 
    ret, img = cam.read() 
    img = cv2.flip(img, 1) # flip video image vertically 
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    faces = face_detector.detectMultiScale(img, 1.3, 5) 

    for (x,y,w,h) in faces: 
        rrec = cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2) 
        #count += 1 
        #dim = (100, 100) 
        #resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
        # Save the captured image into the datasets folder 
        #cv2.imwrite("test1/User." + str(face_id) + '.' + str(count) + ".jpg", img[y:y+h,x:x+w]) 
        crop = img[y:y+h,x:x+w]
        text = recognition(crop)
        cv2.putText(rrec,text,(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)
    cv2.imshow('image', img) 

    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video 
    if k == 27: 
        break 
   

# Do a bit of cleanup 
print("\n [INFO] Exiting and cleanup stuff") 
cam.release() 
cv2.destroyAllWindows()